In [ ]:
import argparse
import hashlib
import json
import numpy as np
import os
import pprint
import tensorflow as tf

import data
import evaluate
import sample
import train
import model as model_pkg
import model_graph
import motion

%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
ntimesteps = 20
frmsz = 241
batchsz = None
output_mode = 'rect_map'
path_data_home = '/home/jvlmdr/projects/2017-01-mem-track/data/'
path_ckpt = './ckpt/'
# dataset = 'OTB-50'
sample_frames_params = {'kind': 'full'}
augment_motion_params = {'kind': 'none'}
random_seed = 0
model_base_params = {'object_centric': True, 'use_heatmap': False}

In [ ]:
dataset = data.OTB('OTB-50',
    frmsz=241,
    path_data=os.path.join(path_data_home, 'OTB'))

In [ ]:
def eval_model_params(model_params):
    tf.reset_default_graph()
    
    model = model_pkg.SimpleSearch(
        ntimesteps=ntimesteps,
        frmsz=frmsz,
        batchsz=None,
        output_mode=output_mode,
        stat={'mean': 0.5, 'std': 1.0},
        weight_decay=1.0, # No effect at test time.
        **model_params)
    with tf.name_scope('input'):
        example = model_graph.make_example_placeholders(
            batchsz=None,
            ntimesteps=ntimesteps,
            frmsz=frmsz)
        run_opts = model_graph.make_option_placeholders()
    prediction_crop, window, prediction, init_state, final_state = model_graph.process_sequence(
        model, example, run_opts,
        batchsz=None, ntimesteps=ntimesteps, frmsz=frmsz)
    restorer = tf.train.Saver()
    
    eval_model = evaluate.Model(
        batch_size=None,
        image_size=(frmsz, frmsz),
        sequence_len=ntimesteps,
        example=example,
        run_opts=run_opts,
        prediction_crop=prediction_crop,
        window=window,
        prediction=prediction,
        init_state=init_state,
        final_state=final_state)
    sequences = sample.epoch(
        dataset=dataset,
        rand=np.random.RandomState(random_seed),
        sample_frames=sample.make_frame_sampler(
            dataset=dataset,
            rand=np.random.RandomState(random_seed),
            ntimesteps=ntimesteps,
            **sample_frames_params),
        augment_motion=motion.make_motion_augmenter(
            rand=np.random.RandomState(random_seed),
            **augment_motion_params),
        max_videos=None,
        max_objects=None)
    
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    with tf.Session(config=config) as sess:
        # Load model from checkpoint.
        model_file = tf.train.latest_checkpoint(path_ckpt)
        if model_file is None:
            raise ValueError('could not find checkpoint')
        restorer.restore(sess, model_file)
        
        return evaluate.evaluate(sess, eval_model, sequences)

In [ ]:
profile_range = ['cosine', 'linear', 'step']
weight_range  = [0.3, 1.0, 3.0, 5.0, 10.0, 30.0]
radius_range  = [0.5, 1.0, 2.0]

def make_model_params(profile, radius, weight):
    return dict(model_base_params,
                motion_penalty_profile=profile,
                motion_penalty_radius=radius,
                motion_penalty_weight=weight)

model_params_list = [
    make_model_params(profile, radius, weight)
    for profile in profile_range
    for weight  in weight_range
    for radius  in radius_range
]

def hash_dict(s):
    return hashlib.sha224(json.dumps(s, sort_keys=True)).hexdigest()[:7]

# model_params_dict = {
#     hash_dict(p): p for p in model_params_list
# }

from helpers import cache_json

results = {}
for p in model_params_list:
    k = hash_dict(p)
    print p
    results[k] = cache_json('results_{}.json'.format(k),
                            lambda: eval_model_params(p))
    print results[k]['iou_mean']

In [ ]:
max_iou_mean = 0.0
maximizers = []

for p in model_params_list:
    k = hash_dict(p)
    iou_mean = results[k]['iou_mean']
    if iou_mean > max_iou_mean:
        max_iou_mean = iou_mean
        maximizers = [p]
    elif iou_mean == max_iou_mean:
        max_iou_mean = iou_mean
        maximizers.append(p)

print max_iou_mean
pprint.pprint(maximizers)

In [ ]:
for profile in profile_range:
    for radius in radius_range:
        ps = [make_model_params(profile, radius, weight) for weight in weight_range]
        ks = [hash_dict(p) for p in ps]
        iou_mean = [results[k]['iou_mean'] for k in ks]
        plt.plot(weight_range, iou_mean)

plt.xscale('log')
plt.xlabel('weight')
plt.ylabel('iou_mean')
plt.grid()
plt.show()

In [ ]:
for profile in profile_range:
    for weight in weight_range:
        ps = [make_model_params(profile, radius, weight) for radius in radius_range]
        ks = [hash_dict(p) for p in ps]
        iou_mean = [results[k]['iou_mean'] for k in ks]
        plt.plot(radius_range, iou_mean)

plt.xscale('log')
plt.xlabel('radius')
plt.ylabel('iou_mean')
plt.grid()
plt.show()